# `groupby()`

The `groupby()` function is a method that belongs to the DataFrame class. It works by splitting data into groups based on specified criteria, applying a function to each group independently, then combining the results into a data structure. When applied to a DataFrame, the function returns a groupby object. This groupby object serves as the foundation for different data manipulation operations, including:
1. Aggregation: Computing summary statistics for each group.
2. Transformation: Applying functions to each group and returning modified data
3. Filtration: Selecting specific groups based on certain conditions
4. Iteration: Iterating over groups or values



In [ ]:
import pandas as pd

clothes = pd.DataFrame({'type': ['pants', 'shirt', 'shirt', 'pants', 'shirt', 'pants'],
                        'color': ['red', 'blue', 'green', 'blue', 'green', 'red'],
                        'price_usd': [20, 35, 50, 40, 100, 75],
                        'mass_g': [125, 440, 680, 200, 395, 485]})


clothes

,type,color,price_usd,mass_g
0,pants,red,20,125
1,shirt,blue,35,440
2,shirt,green,50,680
3,pants,blue,40,200
4,shirt,green,100,395
5,pants,red,75,485


In [5]:
# grouping dataframe by type object

grouped = clothes.groupby('type')
print(grouped)
print(type(grouped))


print(grouped.mean(numeric_only= True))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
       price_usd  mass_g
type                    
pants  45.000000   270.0
shirt  61.666667   505.0


**Note**:

In this example, `groupby()` combined all the items into groups based on their type and returned a DataFrame object containing the mean of each group for each numeric column in the DataFrame.

Beginning in pandas v.1.5.0, numerical aggregation functions like `sum()`, `mean()`, `median()`, etc. must have their `numeric_only` keyword argument set to True when used with a `groupby()` operation or else pandas will throw an error if your data also contains non-numerical columns. 

In [6]:
print(grouped.mean())

TypeError: Could not convert redbluered to numeric

You can create groups on multiple columns

In [7]:
clothes.groupby(['type', 'color']).min()

price_usd  mass_g
type  color                   
pants blue          40     200
      red           20     125
shirt blue          35     440
      green         50     395

To simply return the number of observations there are in each group, use the `size()` method. This will result in a Series object with the relevant information

In [8]:
clothes.groupby(['type', 'color']).size()

type   color
pants  blue     1
       red      2
shirt  blue     1
       green    2
dtype: int64

# Built-in aggregation functions

1. `count()`: The number of **non-null** values in each group.
2. `sum()`: The sum of values in each group
3. `mean()`:The mean of values in each group
4. `median()`: The median of values in each group
5. `min()`: The minimum value in each group
6. `max()`: The maximum value in each group
7. `std()`: The standard deviation of values in each group 
8. `var()`: The variance of values in each group

In [18]:
test_date = pd.DataFrame({'type':['cat','dog','fish','cat','dog','fish','cow','cat','dog','fish','cow'],
                        'size':[12,29,3,50,12,23,3,30,12,3,20],
                        'age':[1,2,2,4,1,10,2,7,1,2,6]
                        })

test_date

,type,size,age
0,cat,12,1
1,dog,29,2
2,fish,3,2
3,cat,50,4
4,dog,12,1
5,fish,23,10
6,cow,3,2
7,cat,30,7
8,dog,12,1
9,fish,3,2


In [19]:
test_date.groupby('type').agg(['count','sum','mean','median','min','max','std','var'])

size                                                        age      \
     count sum       mean median min max        std         var count sum   
type                                                                        
cat      3  92  30.666667   30.0  12  50  19.008770  361.333333     3  12   
cow      2  23  11.500000   11.5   3  20  12.020815  144.500000     2   8   
dog      3  53  17.666667   12.0  12  29   9.814955   96.333333     3   4   
fish     3  29   9.666667    3.0   3  23  11.547005  133.333333     3  14   

                                                    
          mean median min max       std        var  
type                                                
cat   4.000000    4.0   1   7  3.000000   9.000000  
cow   4.000000    4.0   2   6  2.828427   8.000000  
dog   1.333333    1.0   1   2  0.577350   0.333333  
fish  4.666667    2.0   2  10  4.618802  21.333333

# `agg()`

The `agg()` function is useful when you want apply **multiple** functions to dataframe at the same time. Its most important parameters are:
1. `func`: The function to be applied
2. `axis`: The axis over which to apply the function(default=0)

`agg()` function can be used by itself(without `groupby()`). 

In [20]:
clothes[['price_usd', 'mass_g']].agg(['sum','mean'])

,price_usd,mass_g
sum,320.000000,2325.0
mean,53.333333,387.5


You can use different functions are applied to different columns.

In [22]:
clothes.agg({
    'price_usd':'sum',
    'mass_g':['mean', 'median']
})

,price_usd,mass_g
sum,320.0,NaN
mean,NaN,387.5
median,NaN,417.5


The following example applies the sum() and mean() functions across axis 1. In other words, instead of applying the functions down each column, they’re applied over each row.

In [23]:
clothes[['price_usd', 'mass_g']].agg(['sum', 'mean'], axis=1)

,sum,mean
0,145.0,72.5
1,475.0,237.5
2,730.0,365.0
3,240.0,120.0
4,495.0,247.5
5,560.0,280.0


# `groupby()` with `agg()`

The `groupby()` and `agg()` functions are often used together.

In [24]:
clothes.groupby('color').agg({'price_usd': ['mean', 'max'],
                        'mass_g': ['mean', 'max']})

price_usd      mass_g     
           mean  max   mean  max
color                           
blue       37.5   40  320.0  440
green      75.0  100  537.5  680
red        47.5   75  305.0  485

## MultiIndex

MultiIndex is a hierarchical system of dataframe indexing. It enables you to store and manipulate data with any number of dimensions in lower dimensional data structures such as series and dataframes. 

In [25]:
grouped = clothes.groupby(['color', 'type']).agg(['mean', 'min'])
grouped

price_usd     mass_g     
                 mean min   mean  min
color type                           
blue  pants      40.0  40  200.0  200
      shirt      35.0  35  440.0  440
green shirt      75.0  50  537.5  395
red   pants      47.5  20  305.0  125

In [26]:
# If you inspect the row index
grouped.index

MultiIndex([( 'blue', 'pants'),
            ( 'blue', 'shirt'),
            ('green', 'shirt'),
            (  'red', 'pants')],
           names=['color', 'type'])

In [27]:
# about column indices
grouped.columns

MultiIndex([('price_usd', 'mean'),
            ('price_usd',  'min'),
            (   'mass_g', 'mean'),
            (   'mass_g',  'min')],
           )

To perform selection on a dataframe with a MultiIndex, use loc[] selection and put indices in parentheses. Here are some examples on grouped, which is a dataframe with a two-level row index and a two-level column index.

In [28]:
grouped

price_usd     mass_g     
                 mean min   mean  min
color type                           
blue  pants      40.0  40  200.0  200
      shirt      35.0  35  440.0  440
green shirt      75.0  50  537.5  395
red   pants      47.5  20  305.0  125

In [30]:
# To select a first-level (top) column

grouped.loc[:,'price_usd']

mean  min
color type            
blue  pants  40.0   40
      shirt  35.0   35
green shirt  75.0   50
red   pants  47.5   20

In [31]:
# To select a second-level (bottom) column

grouped.loc[:, ('price_usd','min')]

color  type 
blue   pants    40
       shirt    35
green  shirt    50
red    pants    20
Name: (price_usd, min), dtype: int64

In [32]:
# To select first-level (left-most) row

grouped.loc['blue',:]

price_usd     mass_g     
           mean min   mean  min
type                           
pants      40.0  40  200.0  200
shirt      35.0  35  440.0  440

In [33]:
# To select a bottom-level(right-most) row

grouped.loc[('green', 'shirt'), :]

price_usd  mean     75.0
           min      50.0
mass_g     mean    537.5
           min     395.0
Name: (green, shirt), dtype: float64

In [34]:
# To select individual values

grouped.loc[('green', 'shirt'),('price_usd','mean')]

75.0

If you want to remove the row MultiIndex from a groupby result, include as_index=False as a parameter to your `groupby()` statement

In [35]:
clothes.groupby(['color','type'], as_index=False).mean()

,color,type,price_usd,mass_g
0,blue,pants,40.0,200.0
1,blue,shirt,35.0,440.0
2,green,shirt,75.0,537.5
3,red,pants,47.5,305.0
